In [ ]:
!pip install ultralytics
!pip install hyperopt
!pip install -q wandb

In [ ]:
import torch
import os
import wandb
from IPython.display import Image, clear_output  # to display images
from hyperopt import fmin, tpe, hp, Trials
from ultralytics import YOLO

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("wandb_key")
wandb.login(key=wandb_api_key)

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
def objective(params):
    # Define el modelo con los parámetros dados
    model = YOLO().load('yolov8n.pt')
    
    # Entrena el modelo
    model.train(data="/kaggle/input/cbis-ddsm-mias-inbreast-yolo/data.yaml",
                epochs=params["epochs"],
                dropout=params["dropout"],
                lr0=params["lr"],                
                batch=8,
                augment=True,
                imgsz=1600)
    
    # Evalúa el modelo
    metrics = model.val()
    
    # Devuelve la métrica a optimizar
    return -metrics.results_dict["metrics/recall(B)"]

# Define los rangos de los hiperparámetros
space = {
    'epochs': hp.choice('epochs', [20, 50, 100]),
    'dropout': hp.choice('dropout',[0.0, 0.2, 0.4]),
    'lr': hp.loguniform('lr', -6, -1),
}

trials = Trials()

# Realiza la búsqueda de hiperparámetros
best = fmin(objective, space, algo=tpe.suggest, max_evals=15, trials=trials)
print(best)
print(trials.trials)

In [ ]:
metrics.results_dict["metrics/recall(B)"